In [1]:
import seaborn as sns
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import os

IMPORT AND TRANSFORM DATAFRAME

In [13]:
imag_to_plot = []
full_names = []
#import csv
df = pd.read_csv("BASE (566) - CES33- Febrero 2023.csv", sep = ";")

#create N variable for total cases
N = df.shape[0]

#before melting, discard imgs that are not going to be used for the especific plot
selected_columns = imag_to_plot
selected_columns.append('id')

#create dictionary with names of variables
dict_names = dict(zip(imag_to_plot, full_names))

df_selected = df.drop(columns = [col for col in df.columns if col not in selected_columns])

#replace empty values in imag_lucas
if "voto_lucas" in df_selected.columns:
    df_selected.voto_lucas.replace(" ", "Ns/Nc", inplace=True)


#melt df
df_melt=df_selected.melt(id_vars= "id")

#convert value column to a categoric variable for then order the columns

column_order = ["Seguro lo votaría", "Quizás lo votaría", "Nunca lo votaría", "Ns/Nc"]

df_melt["value"] = df_melt["value"].astype("category")
df_melt['value'] = df_melt['value'].cat.reorder_categories(column_order, ordered=True)

CREATE PIVOT TABLE

In [ ]:
func = lambda x: 100*x.count()/N


table = pd.pivot_table(data=df_melt,
                       columns='value',
                       index= 'variable',
                       values='value',          
                       aggfunc=func
)            

table=table.round(1)

table.sort_values(by="Ns/Nc", ascending=False, axis=0, inplace=True)

#rename index
table=table.rename(index=dict_names)
table

CREAR BARRAS APILADAS FORMATO PRESENTACIÓN

In [ ]:
c = ['#A9D18E', '#FFF2CC', '#FFD966', '#D9D9D9']

plt.style.use("seaborn-white")
fig, ax = plt.subplots(figsize =(14,6))
table.plot(ax=ax, kind='barh',stacked=True,rot=90, color=c) #mark_right=True)

ax.set
#yticks
plt.yticks(rotation = 0)
#ylabel
ax.set_ylabel(None)
#legend
plt.legend(fontsize=15, title=None, ncol=6, frameon=False, bbox_to_anchor=(1, -0.1))
plt.xticks([0,20,40,60,80,100])
plt.yticks(fontsize=16)



for rect in ax.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()
    
    # The height of the bar is the data value and can be used as the label
    label_text = f'{width:.1f}'  # f'{width:.2f}' to format decimal values
    
    # ax.text(x, y, text)
    label_x = x + width / 2
    label_y = y + height / 2
    
    # only plot labels greater than given width
    if width > 2.5:
        ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=15)




plt.xlim(0, 100) 


plt.savefig("barras.png", transparent=True)